In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from utils import data_processing
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

# Stock Price Prediction

Beaucoup de méthodes existent déjà pour prédire l'évolution de valeurs sur un marché. Nous allons voir dans la suite de cet atelier un *réseau de neurones récurrents* ou _RNN_.

![caption](rnn.png)

Ces réseaux de neurones ont la particularité de pouvoir faire des prédictions sur des _séries_ ou _données séquentielles_. Ils sont dotés d'une mémoire qui leur permet de prendre en contact des données qui précèdent les données sur lesquels il travaille.

## Observation des données

Avant toutes choses, il est important de prendre connaissance des données avec lesquels nous allons entraîner notre algorithme. Nous utiliserons des relevés de l'indice boursier S&P 500 basé sur 500 grandes sociétés cotées sur les bourses américaines (Standard & Poor's).

La plupart des modèles développés aujourd'hui en _deep learning_ sont des modèles supervisés. C'est à dire qu'ils se "nourrissent" de données labellisées pour apprendre. Cela demande donc une excellente expertise de ces dernières pour que l'IA atteigne son objectif.

In [ ]:
# Nous chargeons les données
data = pd.read_csv('datas/sp500.csv', header=None)
# Affichons les 5 premières données
data.head()

In [ ]:
# Affichons les données sous forme de graphe pour avoir une meilleure compréhension
data.plot(legend=False)

In [ ]:
# Ici un peu de magie pour préparer les données avant de les envoyer dans le réseau
# data = data.as_matrix()
X_train, y_train, X_test, y_test = data_processing(data.as_matrix())

## RNN network

Une fois que nous avons nos données prêtes, il nous faut un modèle :

In [ ]:
model = Sequential()

model.add(LSTM(
    input_dim=1,
    output_dim=50,
    return_sequences=True))

model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))

model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))

model.add(Activation('linear'))

model.compile(loss='mse', optimizer='rmsprop')

## Training

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=512,
    nb_epoch=100,
    validation_split=0.05)

In [ ]:
# Afficher la loss et val_loss

## Prédictions

In [ ]:
# Prédictions sur X_test
predictions = lstm.predict_sequences_multiple(model, X_test, 50, 50)
# Afficher la prédiction

## Comment améliorer les prédictions ?